Demonstrating that the "gradient field" can be viewed irrespective of the underlying parametrisation to guide design principles.

In [1]:
%load_ext autoreload
%autoreload 2

# sweep.ipynb

import numpy as np
from datetime import datetime

from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
from adjoint_sim_sf.Optimiser import Optimiser
from adjoint_sim_sf import Experiment

from SQDMetal.COMSOL.Model import COMSOL_Model
from SQDMetal.COMSOL.SimCapacitance import COMSOL_Simulation_CapMats
from SQDMetal.COMSOL.SimRFsParameter import COMSOL_Simulation_RFsParameters


# 1) COMSOL engine
if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 40,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

# (Optional) deterministic sampling for EPR sources across sweeps
evaluator.random_seed = 12345



In [ ]:
import math
# Default folder: exp_YYYYMMDD-HH%M-%S (comes from Experiment.__init__)
exp = Experiment(name=f"parameter_free_demo{datetime.now().strftime('%Y%m%d-%H%M-%S')}.ipynb")





In [ ]:
params = np.array([0.233, 0.25])
qk_design = evaluator.parametric_designer.build_qk_design(params)

# Run the forward and adjoint calculations, only for jj source
fwd_sparams = evaluator._fwd_calculation(qk_design)
jj_sources = evaluator._get_objective_info(params, "jj")
adj_sparams = evaluator._adjoint_calculation(qk_design, fwd_sparams)

#Extract the "gradient field" for parameter-free visualisation

fwd_E_dict = fwd_sparams.eval_fields_over_mesh()    # returns  {'coords': np.vstack([x,y,z]).T,
                                                    # 'E': np.vstack([Ex,Ey,Ez]).T, ... other fields


meshcoords = fwd_E_dict['coords']  # N x 3 array of xyz coordinates
fwd_E_vals = fwd_E_dict['E']

adj_E_vals = adj_sparams.eval_field_at_pts('E', meshcoords)  # N x 3 array of Ex,Ey,Ez at meshcoords

Enp = np.array(fwd_E_vals)

Vnp = np.array(adj_E_vals)
coordnp = np.array(meshcoords)

phase = np.exp(1j * math.pi / 2) 

# confusingly, need to NOT conjugate Vnp here to match adjoint calc because it is already done during the simulation.
pairwise_ip = np.einsum('ij, ij->i', Vnp * phase, Enp)
import matplotlib.pyplot as plt
import numpy as np

# We are filtering out only the field directly above the transmon.
x_min, x_max = -5e-4, 5e-4  # the outer "ground" metal square.
y_min, y_max = -5e-4, 5e-4
z_min, z_max = 0, 1e-6  # micrometer slice

# Combine all filters
mask = (
    (coordnp[:, 0] >= x_min) & (coordnp[:, 0] <= x_max) &
    (coordnp[:, 1] >= y_min) & (coordnp[:, 1] <= y_max) &
    (coordnp[:, 2] >= z_min) & (coordnp[:, 2] <= z_max)
)

# Apply mask
filtered_coords = coordnp[mask]
filtered_values = np.real(pairwise_ip[mask])

# Multicolor mapping
colors = np.select(
    [
        (filtered_values >= 0) & (filtered_values < 1e20),
        (filtered_values >= 1e20),
        (filtered_values < 0) & (filtered_values > -1e20),
        (filtered_values <= -1e20)
    ],
    ['deepskyblue', 'navy', 'lightcoral', 'darkred'],
    default='gray'
)

# Plot
fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(filtered_coords[:, 0], filtered_coords[:, 1],
           c=colors, s=5)

ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title(rf"$\mathrm{{Re}}(v^\dagger E)$, transmon too big")

plt.show()


In [11]:
import matplotlib.pyplot as plt
import numpy as np

# Define your filter bounds
x_min, x_max = -5e-4, 5e-4
y_min, y_max = -5e-4, 5e-4
z_min, z_max = 0, 1e-6  # micrometer slice

# Combine all filters
mask = (
    (coordnp[:, 0] >= x_min) & (coordnp[:, 0] <= x_max) &
    (coordnp[:, 1] >= y_min) & (coordnp[:, 1] <= y_max) &
    (coordnp[:, 2] >= z_min) & (coordnp[:, 2] <= z_max)
)

# Apply mask
filtered_coords = coordnp[mask]
filtered_values = np.real(pairwise_ip[mask])

# Multicolor mapping
colors = np.select(
    [
        (filtered_values >= 0) & (filtered_values < 1e20),
        (filtered_values >= 1e20),
        (filtered_values < 0) & (filtered_values > -1e20),
        (filtered_values <= -1e20)
    ],
    ['deepskyblue', 'navy', 'lightcoral', 'darkred'],
    default='gray'
)

# Plot
fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(filtered_coords[:, 0], filtered_coords[:, 1],
           c=colors, s=5)

ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title(rf"$\mathrm{{Re}}(v^\dagger E)$, transmon too big")

plt.show()


  offsets = np.asanyarray(offsets, float)



In [10]:
print(filtered_values)

[-6.29555205e+22 -1.00648992e+23 -9.12067821e+22 ... -3.13718986e+18
 -6.80337425e+17 -7.39350001e+18]
